# Cleaning

In [80]:
# Importing

import pandas as pd
import numpy as np
import plotly.express as px

path = "../data/raw/houses_moreinfo.csv"

df = pd.read_csv(path)


In [81]:
df.head()

,id,city,postal_code,district,province,price,type_sale,subtype,sale_type,state_construction,...,swimmingpool,has_balcony,has_terrace,has_attic,has_basement,construction_year,epc,heating,life_annuity,area_total
0,1882546,Wenduine,8420,Oostend,West Flanders,163000.0,HOUSE,CHALET,FOR_SALE,AS_NEW,...,0,NaN,NaN,NaN,NaN,2023.0,NaN,GAS,0,155.0
1,2784938,Heist-aan-Zee,8301,Brugge,West Flanders,108000.0,HOUSE,BUNGALOW,FOR_SALE,AS_NEW,...,0,NaN,NaN,NaN,NaN,2023.0,NaN,GAS,0,170.0
2,4679861,Oostende,8400,Oostend,West Flanders,330000.0,HOUSE,HOUSE,FOR_SALE,GOOD,...,0,NaN,1.0,NaN,NaN,NaN,B,NaN,0,65.0
3,5650273,Dilsen-Stokkem,3650,Maaseik,Limburg,535000.0,HOUSE,APARTMENT_BLOCK,FOR_SALE,NaN,...,0,NaN,NaN,NaN,NaN,1988.0,B,GAS,0,0.0
4,6349237,Corswarem,4257,Waremme,Liège,NaN,HOUSE_GROUP,HOUSE_GROUP,FOR_SALE,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [82]:
# Change name of places to lower
def format_name(x):
    return x.lower()

df["city"] = df.city.apply(format_name)
df["district"] = df.district.apply(format_name)
df["province"] = df.province.apply(format_name)

# Substitute 0 for Nan
df.replace(0, np.nan, inplace=True)




In [83]:
# Drop rows with life_annuity == 1
life_annuity = df.query("life_annuity == 1").index

df.drop(life_annuity, inplace=True)

df.shape

(44908, 33)

In [84]:
# Drop where price is NaN
df.dropna(subset=['price'], inplace=True)

In [85]:
# Drop useless columns
df.drop(['life_annuity', 'has_balcony', "sale_type", "type_sale" ], axis=1, inplace=True)

In [87]:
df.shape

# Drop castles
castles = df["subtype"] == "CASTLE"
df_clean = df[~castles]
df_clean.subtype.value_counts()

subtype
HOUSE                   32451
VILLA                    3864
APARTMENT_BLOCK          2306
MIXED_USE_BUILDING       1847
EXCEPTIONAL_PROPERTY      682
TOWN_HOUSE                506
MANSION                   499
BUNGALOW                  359
FARMHOUSE                 273
COUNTRY_COTTAGE           265
MANOR_HOUSE               178
CHALET                    120
OTHER_PROPERTY            112
Name: count, dtype: int64

# Start exploring

In [92]:
df_clean.shape

(43462, 29)

In [93]:
# Missing values
percent_missing = df_clean.isnull().sum() * 100 / len(df)
percent_missing

id                     0.000000
city                   0.000000
postal_code            0.000000
district               0.000000
province               0.000000
price                  0.000000
subtype                0.000000
state_construction    27.930702
living_area           14.576536
terrace_area          75.194155
garden_area           72.347319
rooms                 73.117044
bedrooms               2.522862
bathrooms             13.896420
livingroom_surface    61.010523
kitchen_surface       64.381232
facades               20.840035
has_garden            66.345756
kitchen               43.814622
furnished             98.614494
fireplace             80.885529
swimmingpool          96.946372
has_terrace           51.082211
has_attic             72.147420
has_basement          60.668168
construction_year     44.735996
epc                   19.629153
heating               32.475530
area_total            11.812417
dtype: float64

In [94]:
def IQR (column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    #quartiles = df[column].quantile([0.25, 0.75])
    iqr = Q3 - Q1
    lowerlimit = Q1 - 1.5*iqr
    upperlimit = Q3 + 1.5*iqr

    return lowerlimit, upperlimit

lowerlimit, upperlimit  = IQR("price")

price_no_outliers = df_clean[df_clean['price'].between(lowerlimit, upperlimit)]

price_no_outliers_q1 = price_no_outliers["price"].quantile(0.25)
price_no_outliers_q3 = price_no_outliers["price"].quantile(0.75)

average_price_no_outliers = price_no_outliers.price.mean()

price_no_outliers.shape




(40511, 29)

In [95]:
def thousants_point(x, n):
    return str('{:,}'.format(round(x), n).replace(',','.'))

text_average = "Avg € " + thousants_point(average_price_no_outliers,0)
text_q1 = "Q1 € " + thousants_point(price_no_outliers_q1,0)
text_q3 = "Q3 € " + thousants_point(price_no_outliers_q3,0)

fig = px.histogram(price_no_outliers, x="price", nbins=40) 
fig.update_layout(bargap=0.2)
fig.add_vline(x=average_price_no_outliers, line_dash = 'dash', line_color = 'firebrick', annotation_text= text_average, annotation_position="top")
fig.add_vline(x=price_no_outliers_q1, line_dash = 'dash', line_color = 'green', annotation_text= text_q1, annotation_position="top left")
fig.add_vline(x=price_no_outliers_q3, line_dash = 'dash', line_color = 'green', annotation_text= text_q3, annotation_position="top right")
fig.update_layout(
    title = "Pricing distribution", 
    title_font=dict(size=20,
                    color='blue',
                    family='Arial'),
    title_x=0.1,  # Title aligned with grid
)
fig.show()


In [169]:

mask_epc = price_no_outliers['epc'].isin(["A++", "A+", "A", "B", "C", "D", "E", "F", "G"])
compare_epc= price_no_outliers[mask_epc]
compare_epc = compare_epc.groupby('epc', as_index=False)[['price','construction_year']].mean().round()

In [171]:
fig = px.bar(compare_epc, x='epc', y='price', color='construction_year', title="Avg price per EPC")
fig.update_xaxes(categoryorder='array', categoryarray= ['A++', 'A+', 'A', 'B', 'C', 'D', 'E', 'G'])
fig.show()

In [123]:

epc_sub= {"A++":8, "A+":7, "A":6,"B":5,"C":4,"D":3,"E":2,"F": 1,"G":0}
state_sub = {"AS_NEW":5,"JUST_RENOVATED":4 , "GOOD":3, "TO_BE_DONE_UP":2, "TO_RENOVATE":1, "TO_RESTORE":0 }
correlation_var= price_no_outliers.replace({"epc":epc_sub, "state_construction": state_sub})
mask = correlation_var['epc'].isin([0,1,2,3,4,5,6,7,8, np.nan])

correlation_var= correlation_var[mask]
correlation_var.state_construction.value_counts()
#correlation_var.epc.value_counts()
#correlation_var.isna().sum()

C:\Users\t_dei\AppData\Local\Temp\ipykernel_58060\2722388209.py:3: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



state_construction
3.0    13092
1.0     5506
5.0     5063
2.0     3545
4.0     1441
0.0      367
Name: count, dtype: int64